# Plotting and data analysis
Using both `Python` and `R`

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import time, sys, pickle
import plotnine

In [2]:
import rpy2
%load_ext rpy2.ipython
%R library(tidyverse)
%R library(viridis)
%R library(gganimate)
%R library(gifski)
%R library(transformr)
%R library(ggridges)

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

R[write to console]: ✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.0     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

R[write to console]: Loading required package: viridisLite



'ggridges','transfor...,'gifski',...,'datasets','methods','base'


### Load in the data

In [ ]:
def data_wrangle(df):
    df = df.groupby(['Municipality','CommuterFraction','MutationChance','Day']).mean().reset_index().drop(['RunID'],axis=1)
    df['I'] = df[['Ia','Ip','Is']].sum(axis=1)
    df.drop(['Ia','Ip','Is'], axis=1)
    df = pd.melt(df, id_vars=['CommuterFraction','MutationChance','Municipality','Day','DailyR'], value_vars=['S','E','I','R','H','ICU','D'], var_name='State', value_name='Count')
    df = df.rename(columns={'DailyR':'R'})
    #df = df.loc[df['CommuterFraction'] <= 0.01]
    return df

In [ ]:
def calculate_means(df):
    means_per_municipality = df.groupby(['Municipality','CommuterFraction','MutationChance','State']).mean().reset_index().drop(['Day'],axis=1)
    means_per_day = df.groupby(['Day','CommuterFraction','MutationChance','State']).mean().reset_index()
    means = means_per_day.groupby(['CommuterFraction','MutationChance']).mean().reset_index().drop(['Day','Count'],axis=1)

    return means_per_municipality, means_per_day, means

In [ ]:
df = data_wrangle(pd.read_csv("results/summaries/combinedResults_mutations.txt"))
means_per_municipality, means_per_day, means = calculate_means(df)

df_seed = data_wrangle(pd.read_csv("results/summaries/combinedResults_mutationsSeed0.005.txt"))
means_per_municipality_seed, means_per_day_seed, means_seed = calculate_means(df_seed)

df_seed1 = data_wrangle(pd.read_csv("results/summaries/combinedResults_mutationsSeed0.01.txt"))
means_per_municipality_seed1, means_per_day_seed1, means_seed1 = calculate_means(df_seed)

df_commute = data_wrangle(pd.read_csv("results/summaries/combinedResults_mutations_commuters.txt"))
means_per_municipality_commute, means_per_day_commute, means_commute = calculate_means(df_commute)

## Combining the dataframes

In [ ]:
means_comb = df.groupby(['Municipality','CommuterFraction','MutationChance','Day','State']).mean().reset_index()
means_comb_seed = df_seed.groupby(['Municipality','CommuterFraction','MutationChance','Day','State']).mean().reset_index()
means_comb_seed1 = df_seed1.groupby(['Municipality','CommuterFraction','MutationChance','Day','State']).mean().reset_index()
means_comb_commute = df_commute.groupby(['Municipality','CommuterFraction','MutationChance','Day','State']).mean().reset_index()

means_comb['Seed'] = 'Trondheim only'
means_comb_seed['Seed'] = 'All municipalities, 0.5%'
means_comb_seed1['Seed'] = 'All municipalities, 1.0%'
means_comb_commute['Seed'] = 'Trondheim, with commuting'

#comb = pd.concat([means_comb, means_comb_seed, means_comb_seed1])
comb = pd.concat([means_comb, means_comb_seed, means_comb_commute])
comb

In [ ]:
comb_means = comb.groupby(['Municipality','CommuterFraction','MutationChance','Seed']).mean().reset_index().drop(['Day','Count'],axis=1)
comb_means

In [ ]:
%%R -i comb_means
comb_means$CommuterFraction = factor(comb_means$CommuterFraction)
comb_means$MutationChance = factor(comb_means$MutationChance)

comb_means %>%
    ggplot(aes(MutationChance, R)) + 
    geom_boxplot(aes(fill=CommuterFraction), alpha=0.5) +
    facet_wrap(~Seed) +
    scale_fill_brewer() +
    scale_color_brewer(palette='Set1') +
    theme_minimal() +
    labs(
        x = 'Mutation Chance',
        y = 'R',
        title = 'Daily R for all municipalities',
        subtitle = 'With two different seeding methods'
    ) +
    ylim(0,10) +
    theme(legend.position="bottom")

In [ ]:
%R ggsave('boxplot_combination.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

### Heatmap of daily R

In [ ]:
%%R -i means
means %>%
    ggplot(aes(x=MutationChance, y=CommuterFraction, fill=R)) + 
    geom_tile() +
    theme_minimal() +
    scale_fill_distiller(type='seq', direction=1)+#palette='YlGnBu') + 
    labs(
        x = 'Mutation Chance',
        y = 'Commuter Fraction',
        title = 'Daily R for each municipality'
    )

In [ ]:
%R ggsave('heatmap.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

In [ ]:
%%R -i means_per_municipality
means_per_municipality %>%
    #filter(R > 0.0) %>%
    #filter(State == 'S') %>%
    ggplot(aes(x=MutationChance, y=CommuterFraction, fill=log(R))) + 
    geom_tile() +
    facet_wrap(~Municipality) +
    theme_minimal() +
    scale_fill_distiller(type='seq', direction=1)+
    labs(
        x = 'Mutation Chance',
        y = 'Commuter Fraction',
        title = 'Daily R for each municipality'
    )

In [ ]:
%R ggsave('heatmap_fw.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

In [ ]:
%%R -i means_per_municipality
means_per_municipality$CommuterFraction = factor(means_per_municipality$CommuterFraction)
means_per_municipality$MutationChance = factor(means_per_municipality$MutationChance)

means_per_municipality %>%
    filter(R < 15) %>%
    filter(State == 'S') %>%
    ggplot(aes(MutationChance, R)) + 
    geom_boxplot(aes(fill=CommuterFraction)) +
    scale_fill_brewer() +
    theme_minimal() +
    labs(
        x = 'Mutation Chance',
        y = 'R',
        title = 'Daily R for all municipalities',
        subtitle = 'Seeded initial run in Trondheim, timed run 30 days'
    ) +
    ylim(0,10) +
    theme(legend.position="bottom")

In [ ]:
%R ggsave('boxplot_seedAll.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

In [ ]:
%%R -i means_per_municipality_seed
means_per_municipality_seed$CommuterFraction = factor(means_per_municipality_seed$CommuterFraction)
means_per_municipality_seed$MutationChance = factor(means_per_municipality_seed$MutationChance)

means_per_municipality_seed %>%
    filter(R < 15) %>%
    filter(State == 'S') %>%
    ggplot(aes(MutationChance, R)) + 
    geom_boxplot(aes(fill=CommuterFraction)) +
    scale_fill_brewer() +
    theme_minimal() +
    labs(
        x = 'Mutation Chance',
        y = 'R',
        title = 'Daily R for all municipalities',
        subtitle = 'Seeded 0.5% in every municipality, run 60 days'
    ) +
    ylim(0,10) +
    theme(legend.position="bottom")

In [ ]:
%R ggsave('boxplot_seedTrondheim10.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

### Mutation chance vs. commuter fraction, facet wrap Municipality

In [ ]:
%%R -i means_per_municipality
means_per_municipality %>%
    filter(R > 0.0) %>%
    filter(State == 'S') %>%
    ggplot(aes(x=MutationChance, y=CommuterFraction, color=log(R), size=log(R))) + 
    geom_point(alpha=0.6) +
    theme_minimal() +
    facet_wrap(~Municipality) + 
    scale_colour_viridis_c(guide = 'legend') +
    #scale_size_continuous(range=c(-2,4)) + 
    labs(
        x = 'Mutation Chance',
        y = 'Commuter Fraction',
        title = 'Daily R for each municipality'
    )

In [ ]:
%R ggsave('mutationchance_vs_commuterfrac_fw_municipality.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

### Mutation chance vs. commuter fraction, without facet wrap

In [ ]:
%%R -i means
means %>%
    filter(R > 0.0) %>%
    #filter(State == 'S') %>%
    ggplot(aes(x=MutationChance, y=CommuterFraction, color=log(R), size=log(R))) + 
        geom_point(alpha=0.6) +
        theme_minimal() +
        scale_colour_viridis_c(guide = 'legend') +
        scale_size_continuous() + 
        labs(
            x = 'Mutation Chance',
            y = 'Commuter Fraction',
            title = 'Mean daily R'
        )

In [ ]:
%R ggsave('mutationchance_vs_commuterfrac.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

# Animated plots of daily R

### Daily R with and without faceting for municipalities

In [ ]:
%%R -i df

animatedPlot <- df %>%
    filter(State == 'S') %>%
    ggplot(aes(x=MutationChance, y=CommuterFraction, size=log(R), color=log(R))) + 
    geom_point(alpha=0.6) +
    theme_minimal() +
    scale_colour_viridis_c(guide = 'legend') +
    scale_size_continuous(range = c(-2.5, 5)) + 
    guides(color=guide_legend(), size=guide_legend()) +
    labs(
        x = 'Mutation Chance',
        y  = 'Commuter Fraction',
        title = 'Daily R',
        subtitle = 'R for day: {frame_time}/30'
    ) +
    transition_time(Day) +
    ease_aes('linear') +
    enter_fade() + 
    exit_shrink()

#animate(animatedPlot, duration = 7, fps = 4, width = 800, height = 600,renderer = gifski_renderer())
#anim_save('mutationchance_vs_commuterfrac.gif', path='results/plots')

In [ ]:
%%R 
animatedPlotFacet <- animatedPlot + 
    facet_wrap(~Municipality)
#animate(animatedPlotFacet, duration = 7, fps = 4, width = 800, height = 600, renderer = gifski_renderer())
#anim_save('mutationchance_vs_commuterfrac_fw_municipality.gif', path='results/plots')

# Adding population data to the data frame
Population size, area, and derivatives of these two

In [ ]:
data = df.merge(municipality_data, left_on='Municipality', right_on='Name').drop(['Name','Number','County'],axis=1)
data['PopDensity'] = data['Population']/data['Area']
data['CountPerPop'] = data['Count']/data['Population']
data['CountPerPopDensity'] = data['Count']/data['PopDensity']
data

## SIR plots
With no commuting and mutations

In [ ]:
%%R -i data
data %>%
    filter(State %in% c('E','I','R','D')) %>% 
    filter(CommuterFraction == 0.0 & MutationChance == 0.0) %>%
    ggplot(aes(x=Day, y=log(Count), color=State)) +
    geom_line(alpha=0.8) + 
    facet_wrap(~Municipality) +
    theme_minimal() + 
    scale_colour_brewer(palette = "Set1") + 
    labs(
        x  = 'Day',
        y = 'log(Count)',
        title = 'SIR for Trondheim region'
    )

In [ ]:
%R ggsave('Trondheim_SIR.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

Same, but with numbers divided by population size

In [ ]:
%%R -i data
data %>%
    filter(State %in% c('E','I','R','D')) %>% 
    filter(CommuterFraction == 0.0 & MutationChance == 0.0) %>%
    ggplot(aes(x=Day, y=CountPerPop, color=State)) +
    geom_line(alpha=0.8) + 
    facet_wrap(~Municipality) +
    theme_minimal() + 
    scale_colour_brewer(palette="Set1") + 
    labs(
        x  = 'Day',
        y = 'Count / Population',
        title = 'SIR for Trondheim region',
        subtitle = 'Divided by population size'
    )

In [ ]:
%R ggsave('Trondheim_SIR_perPop.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

In [ ]:
%%R -i data
data %>%
    filter(State %in% c('E','I','R','D')) %>% 
    filter(CommuterFraction == 0.0 & MutationChance == 0.0) %>%
    ggplot(aes(x=Day, y=log(CountPerPopDensity), color=State)) +
    geom_line(alpha=0.8) + 
    facet_wrap(~Municipality) +
    theme_minimal() + 
    scale_colour_brewer(palette="Set1") + 
    labs(
        x  = 'Day',
        y = 'Count / Population density',
        title = 'SIR for Trondheim region',
        subtitle = 'Divided by population density'
    )

In [ ]:
%R ggsave('Trondheim_SIR_perPopDensity.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

In [ ]:
%%R -i data
data %>%
    #filter(Municipality=='Trondheim') %>%
    filter(State=='S') %>%
    ggplot(aes(x=log(PopDensity), y=log(R), color=MutationChance)) + 
    geom_jitter(alpha=0.2) +
    facet_wrap(~Day) + 
    theme_minimal() +
    scale_colour_viridis_c(guide = 'legend') +
    scale_size_continuous(range = c(-2, 5)) + 
    guides(color=guide_legend(), size=guide_legend())

## Histogram

In [ ]:
%%R -i data
data %>%
    ggplot(aes(x = Day, y=log(R), colour=log(R), size=log(R))) +  
    geom_point(alpha=0.2) + 
    facet_wrap(~Municipality) + 
    theme_minimal() +
    scale_colour_viridis()

In [ ]:
mean = means_commute.groupby(['MutationChance','CommuterFraction']).mean().reset_index()
cont = plt.tricontourf(mean['MutationChance'], mean['CommuterFraction'], mean['R'])
cbar = plt.colorbar(cont, shrink=0.8)
cbar.ax.set_ylabel('Daily R')
plt.xlabel('Mutation Chance')
plt.ylabel('Commuter prevalence')
plt.savefig('results/plots/contour_plot_mutations.png', dpi=500)

In [ ]:
df

In [ ]:
%%R -i df
df %>%
    ggplot(aes(x=Day, y=log(R), colour=log(R))) +  
    geom_boxplot() + 
    facet_wrap(~Municipality) + 
    theme_minimal()

# Python plots
To be made into automated functions later. Probably...

In [ ]:
df = pd.read_csv("results/summaries/combinedResults_mutationsSeed0.005.txt")
#df = pd.read_csv("results/summaries/combinedResults_mutations.txt")
df = df.groupby(['Municipality','CommuterFraction','MutationChance','Day']).mean().reset_index().drop(['RunID'],axis=1)
df['I'] = df[['Ia','Ip','Is']].sum(axis=1)
df.drop(['Ia','Ip','Is'], axis=1)
df = pd.melt(df, id_vars=['CommuterFraction','MutationChance','Municipality','Day','DailyR'], value_vars=['S','E','I','R','H','ICU','D'],var_name='State',value_name='Count')
df = df.rename(columns={'DailyR':'R'})
df

In [ ]:
means = df.groupby(['CommuterFraction','MutationChance']).mean().reset_index().drop(['Day','Count'],axis=1)
means.head()

In [ ]:
m = means.pivot("CommuterFraction", "MutationChance", "R")
sns.heatmap(m,cmap="YlGnBu");

In [ ]:
means = df.groupby(['CommuterFraction','MutationChance']).mean().reset_index().drop(['Day','Count'],axis=1)
sns.boxplot(x="MutationChance", y="R",hue="CommuterFraction",data=df);

# Looking at peaks and timing

In [ ]:
def findDayOfMaxR(df):
    data = df.drop(['State','Count'], axis=1)
    data = data.loc[data.groupby(['CommuterFraction','MutationChance','Municipality'])['R'].idxmax()]
    return data

In [ ]:
df = data_wrangle(pd.read_csv("results/summaries/combinedResults_mutations.txt")).dropna(axis=0)
df
#df = findDayOfMaxR(df)

In [ ]:
comb.dropna(axis=0)

In [ ]:
comb_day = findDayOfMaxR(comb.dropna(axis=0))
comb_day

In [ ]:
%%R -i comb_day
peak_R_fw <- comb_day %>%
filter(Day < 30) %>%
    ggplot(aes(x=Day, y=R, color=Seed)) +
    geom_point(alpha=0.4) +
    facet_wrap(~Municipality) +
    ylim(0,50) + 
    theme_minimal() +
    scale_colour_brewer(type='qual', palette='Set1') +
    labs(
        title='Day of peak R',
        y = 'Peak R value',
        x = 'Day of highest R'
    )
peak_R_fw

In [ ]:
%R ggsave('peak_R_fw.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

In [ ]:
%%R
peak_R_fw_trendline <- peak_R_fw + 
    geom_smooth(formula=y~x, method='loess', alpha=0.5)
peak_R_fw_trendline

In [ ]:
%R ggsave('peak_R_fw_trendline.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

In [ ]:
%%R -i comb_day

peak_R <- comb_day %>%
filter(Day < 30 & Day > 1) %>%
    ggplot(aes(x=Day, y=R, color=Seed)) +
    geom_point(alpha=0.4) +
    #geom_smooth(formula=y~x, method='lm', alpha=0.5) +
    ylim(0,50) + 
    theme_minimal() +
    scale_colour_brewer(type='qual', palette='Set1') +
    labs(
        title='Day of peak R',
        y = 'Peak R value',
        x = 'Day of highest R'
    )
peak_R

In [ ]:
%R ggsave('peak_R.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

In [ ]:
%%R
peak_R_trendline <- peak_R + 
    geom_smooth(formula=y~x, method='loess', alpha=0.5)
peak_R_trendline

In [ ]:
%R ggsave('peak_R_trendline.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

In [ ]:
%%R -i comb_day
comb_day$CommuterFraction <- factor(comb_day$CommuterFraction)
comb_day$MutationChance <- factor(comb_day$MutationChance)

comb_day %>%
    filter(MutationChance==0.0) %>%
    filter(Day < 30 & Day > 1) %>%
    #filter(CommuterFraction==0.0 & MutationChance==0.0) %>%
    ggplot(aes(x=Day, y=Municipality, color=Seed, size=R)) +
    geom_point(alpha=0.6) +
    facet_wrap(~CommuterFraction) +
    theme_minimal()

In [ ]:
%R ggsave('whatever_this_is.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

In [ ]:
%%R -i comb_day

density_ridge <- comb_day %>%
    #filter(MutationChance==0.0) %>%
    #filter(CommuterFraction==0.0 & MutationChance==0.0) %>%
    ggplot(aes(x=Day, y=Municipality, fill=Seed)) +
    geom_density_ridges(alpha=0.4, rel_min_height=0.05, bandwidth = 1) +
    theme_ridges(grid = FALSE) +
    scale_fill_brewer(type='qual', palette='Set1') +
    theme_minimal() +
    labs(
        x = 'Day of peak R',
        y = 'Municipality',
        title = 'Day of peak R value',
        subtitle = 'For all municipalities'
    ) +
    xlim(1,30) +
    theme(legend.position="bottom")
density_ridge

In [ ]:
%R ggsave('density_ridge_R_day.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

In [ ]:
%%R 
density_ridge + facet_grid(MutationChance~CommuterFraction)

In [ ]:
dat = comb#comb[comb['State']=='E']
comb_State = dat.loc[dat.groupby(['CommuterFraction','MutationChance','Municipality','State'])['Count'].idxmax()]
comb_State


In [ ]:
%%R -i comb_State

comb_State %>%
    filter(State %in% c('E','I','H','ICU')) %>% 
    #filter(CommuterFraction==0.0 & MutationChance==0.0) %>%
    ggplot(aes(x=Day, y=Municipality, fill=Seed)) +
    geom_density_ridges(alpha=0.4, rel_min_height=0.05, bandwidth = 1) +
    scale_fill_brewer(type='qual', palette='Set1') +
    theme_ridges(grid = FALSE) +
    theme_minimal() +
    facet_wrap(~State, scales='free_y') + 
    #facet_grid(MutationChance~CommuterFraction, labeller = label_both) +
    labs(
        x = 'Day of peak',
        y = 'Municipality',
        title = 'Day of peak for each state',
        subtitle = 'For all municipalities',
        fill = 'Seed method'
    ) +
    theme(
        axis.text.y = element_text(size=5),
        legend.position="bottom"
    )

In [ ]:
%R ggsave('density_ridge_state_day.png', plot = last_plot(), width = 8, height = 6, dpi=300, path='results/plots')

## Plot of base simulations parameters
No commuting and no mutations, seeded in all municipalitites

In [3]:
df = pd.read_csv('results/summaries/combinedResults_mutations_commutersSeed0.001.txt')
df['I'] = df[['Ia','Ip','Is']].sum(axis=1)
df.drop(['Ia','Ip','Is'], axis=1)
df = pd.melt(df, id_vars=['CommuterFraction','MutationChance','Municipality','Day','DailyR', 'RunID'], value_vars=['S','E','I','R','H','ICU','D'], var_name='State', value_name='Count')
df = df.rename(columns={'DailyR':'R'})
df
#mean = df.groupby(['Municipality','CommuterFraction','MutationChance','Day']).mean().reset_index().drop(['RunID'],axis=1)
df.dropna(inplace=True)
mean_R = df.groupby(['Municipality','CommuterFraction','MutationChance','Day']).agg({'R':['mean','std']})
mean_state = df.groupby(['Municipality','CommuterFraction','MutationChance','Day','State']).agg({'R':['mean','std'], 'Count':['mean','std']})
mean_R

R          
                                                      mean       std
Municipality CommuterFraction MutationChance Day                    
Flatanger    0.0              0.0            2    0.000000  0.000000
                                             3    7.721286  9.008313
                                             4    5.151286  6.635569
                                             5    1.290000  2.060806
                                             6    0.968857  2.397799
...                                                    ...       ...
Ørland       1.0              1.0            56   1.008857  0.218969
                                             57   0.897571  0.399197
                                             58   0.725571  0.376879
                                             59   1.008000  0.334900
                                             60   0.775714  0.301610

[39645 rows x 2 columns]

In [77]:
#['_'.join(col).strip() for col in mean_R.columns.values]
mean_R.columns = mean_R.columns.to_flat_index()
#stats.columns.get_level_values(0)
#stats.reset_index()
#stats['R']['std']

In [4]:
#%%R -i mean_R
#mean_R

,CommuterFraction,MutationChance,Municipality,Day,R,RunID,State,Count
1,0.0,0.0,Trondheim,2,3.299,1,S,208349
2,0.0,0.0,Trondheim,3,3.497,1,S,208290
3,0.0,0.0,Trondheim,4,2.853,1,S,208227
4,0.0,0.0,Trondheim,5,2.263,1,S,208167
5,0.0,0.0,Trondheim,6,2.705,1,S,208080
...,...,...,...,...,...,...,...,...
2204995,1.0,1.0,Åfjord,56,1.241,7,D,26
2204996,1.0,1.0,Åfjord,57,0.777,7,D,28
2204997,1.0,1.0,Åfjord,58,0.870,7,D,29
2204998,1.0,1.0,Åfjord,59,1.166,7,D,29
